# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 18 2022 12:34PM,248869,19,8663618,VITABIOTICS USA INC.,Released
1,Oct 18 2022 12:32PM,248868,10,PRF-39496,Bio-PRF,Released
2,Oct 18 2022 12:19PM,248867,10,PRF-39493,Bio-PRF,Released
3,Oct 18 2022 12:16PM,248866,10,CLI00405.1,"CLINUVEL, Inc.",Released
4,Oct 18 2022 12:16PM,248866,10,CLI00404.1,"CLINUVEL, Inc.",Executing
5,Oct 18 2022 11:56AM,248865,22,630930,"NBTY Global, Inc.",Released
6,Oct 18 2022 11:54AM,248864,19,8663102,VITABIOTICS USA INC.,Released
7,Oct 18 2022 11:51AM,248863,22,630934,"NBTY Global, Inc.",Released
8,Oct 18 2022 11:38AM,248862,10,WT5386 Out,Eywa Pharma Inc.,Released
9,Oct 18 2022 11:38AM,248861,19,ADV80006632,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,248866,Executing,1
26,248866,Released,1
27,248867,Released,1
28,248868,Released,1
29,248869,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248865,NaN,NaN,1.0
248866,NaN,1.0,1.0
248867,NaN,NaN,1.0
248868,NaN,NaN,1.0
248869,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248636,0.0,1.0,0.0
248773,0.0,0.0,1.0
248813,0.0,0.0,1.0
248836,0.0,39.0,6.0
248837,0.0,0.0,37.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248636,0,1,0
248773,0,0,1
248813,0,0,1
248836,0,39,6
248837,0,0,37


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248636,0,1,0
1,248773,0,0,1
2,248813,0,0,1
3,248836,0,39,6
4,248837,0,0,37


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248636,,1,
1,248773,,,1
2,248813,,,1
3,248836,,39,6
4,248837,,,37


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 18 2022 12:34PM,248869,19,VITABIOTICS USA INC.
1,Oct 18 2022 12:32PM,248868,10,Bio-PRF
2,Oct 18 2022 12:19PM,248867,10,Bio-PRF
3,Oct 18 2022 12:16PM,248866,10,"CLINUVEL, Inc."
5,Oct 18 2022 11:56AM,248865,22,"NBTY Global, Inc."
6,Oct 18 2022 11:54AM,248864,19,VITABIOTICS USA INC.
7,Oct 18 2022 11:51AM,248863,22,"NBTY Global, Inc."
8,Oct 18 2022 11:38AM,248862,10,Eywa Pharma Inc.
9,Oct 18 2022 11:38AM,248861,19,"AdvaGen Pharma, Ltd"
35,Oct 18 2022 11:34AM,248859,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 18 2022 12:34PM,248869,19,VITABIOTICS USA INC.,,,1
1,Oct 18 2022 12:32PM,248868,10,Bio-PRF,,,1
2,Oct 18 2022 12:19PM,248867,10,Bio-PRF,,,1
3,Oct 18 2022 12:16PM,248866,10,"CLINUVEL, Inc.",,1,1
4,Oct 18 2022 11:56AM,248865,22,"NBTY Global, Inc.",,,1
5,Oct 18 2022 11:54AM,248864,19,VITABIOTICS USA INC.,,,1
6,Oct 18 2022 11:51AM,248863,22,"NBTY Global, Inc.",,,1
7,Oct 18 2022 11:38AM,248862,10,Eywa Pharma Inc.,,,1
8,Oct 18 2022 11:38AM,248861,19,"AdvaGen Pharma, Ltd",,6,20
9,Oct 18 2022 11:34AM,248859,10,Eywa Pharma Inc.,,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 12:34PM,248869,19,VITABIOTICS USA INC.,1,,
1,Oct 18 2022 12:32PM,248868,10,Bio-PRF,1,,
2,Oct 18 2022 12:19PM,248867,10,Bio-PRF,1,,
3,Oct 18 2022 12:16PM,248866,10,"CLINUVEL, Inc.",1,1,
4,Oct 18 2022 11:56AM,248865,22,"NBTY Global, Inc.",1,,
5,Oct 18 2022 11:54AM,248864,19,VITABIOTICS USA INC.,1,,
6,Oct 18 2022 11:51AM,248863,22,"NBTY Global, Inc.",1,,
7,Oct 18 2022 11:38AM,248862,10,Eywa Pharma Inc.,1,,
8,Oct 18 2022 11:38AM,248861,19,"AdvaGen Pharma, Ltd",20,6,
9,Oct 18 2022 11:34AM,248859,10,Eywa Pharma Inc.,14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 12:34PM,248869,19,VITABIOTICS USA INC.,1,,
1,Oct 18 2022 12:32PM,248868,10,Bio-PRF,1,,
2,Oct 18 2022 12:19PM,248867,10,Bio-PRF,1,,
3,Oct 18 2022 12:16PM,248866,10,"CLINUVEL, Inc.",1,1,
4,Oct 18 2022 11:56AM,248865,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 12:34PM,248869,19,VITABIOTICS USA INC.,1.0,NaN,NaN
1,Oct 18 2022 12:32PM,248868,10,Bio-PRF,1.0,NaN,NaN
2,Oct 18 2022 12:19PM,248867,10,Bio-PRF,1.0,NaN,NaN
3,Oct 18 2022 12:16PM,248866,10,"CLINUVEL, Inc.",1.0,1.0,NaN
4,Oct 18 2022 11:56AM,248865,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 12:34PM,248869,19,VITABIOTICS USA INC.,1.0,0.0,0.0
1,Oct 18 2022 12:32PM,248868,10,Bio-PRF,1.0,0.0,0.0
2,Oct 18 2022 12:19PM,248867,10,Bio-PRF,1.0,0.0,0.0
3,Oct 18 2022 12:16PM,248866,10,"CLINUVEL, Inc.",1.0,1.0,0.0
4,Oct 18 2022 11:56AM,248865,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2239703,23.0,15.0,0.0
15,995364,64.0,39.0,0.0
19,1244295,25.0,6.0,0.0
20,746466,3.0,5.0,9.0
21,248636,0.0,1.0,0.0
22,497728,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2239703,23.0,15.0,0.0
1,15,995364,64.0,39.0,0.0
2,19,1244295,25.0,6.0,0.0
3,20,746466,3.0,5.0,9.0
4,21,248636,0.0,1.0,0.0
5,22,497728,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,23.0,15.0,0.0
1,15,64.0,39.0,0.0
2,19,25.0,6.0,0.0
3,20,3.0,5.0,9.0
4,21,0.0,1.0,0.0
5,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,23.0
1,15,Released,64.0
2,19,Released,25.0
3,20,Released,3.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,0.0,0.0,0.0,9.0,0.0,0.0
Executing,15.0,39.0,6.0,5.0,1.0,0.0
Released,23.0,64.0,25.0,3.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,0.0,0.0,0.0,9.0,0.0,0.0
1,Executing,15.0,39.0,6.0,5.0,1.0,0.0
2,Released,23.0,64.0,25.0,3.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,0.0,0.0,0.0,9.0,0.0,0.0
1,Executing,15.0,39.0,6.0,5.0,1.0,0.0
2,Released,23.0,64.0,25.0,3.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()